## Automated anisotropic resistivity inversion for efficient formation evaluation and uncertainty quantification

### Misael M. Morales, Ali Eghbali, Oriyomi Raheem, Michael Pyrcz, Carlos Torres-Verdin
***
## Gradient-Based Inversion
***

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lasio
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from scipy import linalg, optimize
from numdifftools import Jacobian, Hessian

my_box = dict(facecolor='lightgrey', edgecolor='k', alpha=0.5)

In [ ]:
def plot_curve(ax, df, curve, lb=None, ub=None, color='k', pad=0, s=2, mult=1,
            units:str=None, mask=None, offset:int=0, title:str=None, label:str=None,
            semilog:bool=False, bar:bool=False, fill:bool=None, rightfill:bool=False,
            marker=None, edgecolor=None, ls=None, alpha=None):
        if mask is None:
            x, y = -offset+mult*df[curve], df.index
        else:
            x, y = -offset+mult*df[curve][mask], df.index[mask]
        lb = x[~np.isnan(x)].min() if lb is None else lb
        ub = x[~np.isnan(x)].max() if ub is None else ub
        if semilog:
            ax.semilogx(x, y, c=color, label=curve, alpha=alpha,
                        marker=marker, markersize=s, markeredgecolor=edgecolor, linestyle=ls, linewidth=s)
        else:
            if bar:
                ax.barh(y, x, color=color, label=curve, alpha=alpha)
            else:
                ax.plot(x, y, c=color, label=curve, alpha=alpha,
                        marker=marker, markersize=s, markeredgecolor=edgecolor, linewidth=s, linestyle=ls)
        if fill:
            if rightfill:
                ax.fill_betweenx(y, x, ub, alpha=alpha, color=color)
            else:
                ax.fill_betweenx(y, lb, x, alpha=alpha, color=color)
        if units is None:
            if hasattr(df, 'curvesdict'):
                units = df.curvesdict[curve].unit
            else:
                units = ''
        ax.set_xlim(lb, ub)
        ax.grid(True, which='both')
        ax.set_title(title, weight='bold') if title != None else None
        xlab = label if label is not None else curve
        if offset != 0:
            ax.set_xlabel('{} [{}] with {} offset'.format(xlab, units, offset), color=color, weight='bold')
        else:
            ax.set_xlabel('{} [{}]'.format(xlab, units), color=color, weight='bold')
        ax.xaxis.set_label_position('top'); ax.xaxis.set_ticks_position('top')
        ax.xaxis.set_tick_params(color=color, width=s)
        ax.spines['top'].set_position(('axes', 1+pad/100))
        ax.spines['top'].set_edgecolor(color); ax.spines['top'].set_linewidth(2)
        if ls is not None:
            ax.spines['top'].set_linestyle(ls)
        return None

In [ ]:
def quadratic_inversion(df, Rvsh=None, Rhsh=None):
    quad_inv = []
    for _, row in df.iterrows():
        Rv, Rh = row['Rv'], row['Rh']
        Rvsh, Rhsh = row['Rvsh'], row['Rhsh']
        a = Rh*Rvsh - Rh*Rhsh
        b = Rv**2 + Rvsh*Rhsh - 2*Rh*Rhsh
        c = Rv*Rhsh - Rh*Rhsh
        qsol = np.roots([a,b,c])
        if len(qsol) == 1:
            quad_inv.append({'Rss_q':qsol[0], 'Csh_q':np.nan})
        elif len(qsol) == 2:
            quad_inv.append({'Rss_q':qsol[0], 'Csh_q':qsol[1]})
        else:
            quad_inv.append({'Rss_q':np.nan, 'Csh_q':np.nan})
    return pd.DataFrame(quad_inv, index=df.index)

In [ ]:
def error_metrics(df):
    mse_rv = mean_squared_error(df['Rv'], df['Rv_sim'])
    mse_rh = mean_squared_error(df['Rh'], df['Rh_sim'])
    r2_rv = r2_score(df['Rv'], df['Rv_sim'])*100
    r2_rh = r2_score(df['Rh'], df['Rh_sim'])*100
    sterr_rv = np.mean(np.abs(df['Rv']-df['Rv_sim'])) / np.std(np.abs(df['Rv']-df['Rv_sim']))
    sterr_rh = np.mean(np.abs(df['Rh']-df['Rh_sim'])) / np.std(np.abs(df['Rh']-df['Rh_sim']))
    mape_rv = mean_absolute_percentage_error(df['Rv'], df['Rv_sim']) * 100
    mape_rh = mean_absolute_percentage_error(df['Rh'], df['Rh_sim']) * 100
    print('Mean Squared Error - Rv: {:.4f}  | Rh: {:.4f}'.format(mse_rv, mse_rh))
    print('R2 Score           - Rv: {:.3f}  | Rh: {:.3f}'.format(r2_rv, r2_rh))
    print('Standard Error     - Rv: {:.4f}  | Rh: {:.4f}'.format(sterr_rv, sterr_rh))
    print('MAPE               - Rv: {:.3f}%  | Rh: {:.3f}%'.format(mape_rv, mape_rh))
    return None

In [ ]:
def plot_crossplot(sol, figsize=(10,4), cmap='jet', alpha=0.66, vlim:tuple=(0.2,100), hlim:tuple=(0.25,10), axlim:list=[0.1,100]):
    fig, axs = plt.subplots(1, 3, figsize=figsize, width_ratios=[1,1,0.1])
    ax1, ax2, cax = axs
    im1 = ax1.scatter(sol['Rv'], sol['Rv_sim'], c=sol.index, alpha=alpha, cmap=cmap)
    im2 = ax2.scatter(sol['Rh'], sol['Rh_sim'], c=sol.index, alpha=alpha, cmap=cmap)
    r2_rv = r2_score(sol['Rv'], sol['Rv_sim'])*100
    r2_rh = r2_score(sol['Rh'], sol['Rh_sim'])*100
    for i, ax in enumerate([ax1, ax2]):
        ax.plot(axlim, axlim, 'k--')
        ax.set(xscale='log', yscale='log', xlabel='Measured', ylabel='Simulated', title=['$R_v$', '$R_h$'][i])
        ax.grid(True, which='both')
        ax.text(3, 1.25, ['$R^2$: {:.2f}%'.format(r2_rv), '$R^2$: {:.2f}%'.format(r2_rh)][i], bbox=my_box)
    ax1.set(xlim=vlim, ylim=vlim)
    ax2.set(xlim=hlim, ylim=hlim)
    cb = plt.colorbar(im1, cax=cax); cb.set_label('Depth [ft]', rotation=270, labelpad=15)
    plt.tight_layout()
    plt.show()
    return None

***
## Field Dataset 1

In [ ]:
# load dataset
well1 = lasio.read('cases/well1.las').df()
well1_cols = ['CALI','AT10','AT30','AT60','AT90','GR','PE','TNPH','RHOZ','PHID_SS','RV72H_1D_FLT','RH72H_1D_FLT']
well1 = well1[well1_cols]

In [ ]:
# rename columns to standard
well1_newcols = ['CALI','AT10','AT30','AT60','AT90','GR','PEF','NPHI','RHOZ','PHID_SS','Rv','Rh']
well1.columns = well1_newcols

In [ ]:
# define baseline shale parameters
well1.loc[:,'Rvsh'] = 2.813
well1.loc[:,'Rhsh'] = 0.775

In [ ]:
# define starting and ending depth potints
zstart = int(np.argwhere(well1.index == 9720).squeeze())
zend   = int(np.argwhere(well1.index == 10110).squeeze())
well1  = well1.iloc[zstart:zend]
well1 = well1.dropna()

In [ ]:
# visualize dataset
fig, axs = plt.subplots(1, 5, figsize=(10,10), sharey=True)
ax1a, ax2a, ax3a, ax4a, ax5a = axs

ax1b = ax1a.twiny()
plot_curve(ax1a, well1, 'CALI', units='in', lb=0, ub=48, color='dodgerblue', alpha=0.33, fill=True)
plot_curve(ax1b, well1, 'GR', units='API', lb=0, ub=150, color='green', pad=8)

ax2b = ax2a.twiny()
plot_curve(ax2a, well1, 'NPHI', units='v/v', lb=0.6, ub=0.0, color='blue')
plot_curve(ax2b, well1, 'RHOZ', units='g/cm3', lb=1.65, ub=2.65, color='r', pad=8)

ax3b, ax3c, ax3d = ax3a.twiny(), ax3a.twiny(), ax3a.twiny()
plot_curve(ax3a, well1, 'AT10', units='ohm.m', lb=0.2, ub=100, color='b', semilog=True)
plot_curve(ax3b, well1, 'AT30', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=8)
plot_curve(ax3c, well1, 'AT60', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=16)
plot_curve(ax3d, well1, 'AT90', units='ohm.m', lb=0.2, ub=100, color='r', semilog=True, pad=24)

plot_curve(ax4a, well1, 'PEF', units='b/e', lb=2, ub=5, color='purple')

ax5b = ax5a.twiny()
plot_curve(ax5a, well1, 'Rv', units='ohm.m', lb=0.2, ub=100, color='darkred', semilog=True)
plot_curve(ax5b, well1, 'Rh', units='ohm.m', lb=0.2, ub=100, color='darkblue', semilog=True, pad=8)

ax1a.invert_yaxis()
ax1a.set_ylabel('Depth [ft]', weight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# linear (csh only) and quadratic inversion
well1['Csh_lin'] = (well1['GR'] - well1['GR'].min()) / (well1['GR'].max() - well1['GR'].min())
qinv = np.abs(quadratic_inversion(well1))
well1 = well1.join(qinv)

In [ ]:
# moving window shale properties
well1['Rvsh_win'] = np.nan_to_num(np.array(pd.Series(well1['Rv']).rolling(250).max().tolist()), nan=well1['Rv'].mean())
well1['Rhsh_win'] = np.nan_to_num(np.array(pd.Series(well1['Rh']).rolling(250).max().tolist()), nan=well1['Rh'].mean())
R_min = np.min([well1['Rh'].min(), well1['Rv'].min()])
R_max = np.max([well1['Rh'].max(), well1['Rv'].max()])

In [ ]:
# gradient-based inversion
grmax, calimax = well1['GR'].max(), well1['CALI'].max()
lambda_reg = 1e-5
possible_methods = ['L-BFGS-B', 'TNC', 'SLSQP', 'trust-constr']

In [ ]:
def objective(variables, *args):
    Csh, Rs = variables   
    Rv, Rh, Rvsh, Rhsh = args[0], args[1], args[2], args[3]
    grw, caliw = args[4]/grmax, args[5]/calimax
    
    def loss():
        eq1 = (Csh*Rvsh + (1-Csh)*Rs) - Rv
        eq2 = (Csh/Rhsh + (1-Csh)/Rs) - (1/Rh)
        return np.expand_dims(np.array([eq1, eq2]),-1)
    
    def weighting(method='gr'):
        if method=='gr':
            Wd1, Wd2 = 1/Rv/grw, 1*Rh/grw
        elif method=='cali':
            Wd1, Wd2 = 1/Rv/caliw, 1*Rh/caliw
        elif method=='data':
            Wd1, Wd2 = 1/Rv, 1*Rh
        elif method=='none':
            Wd1, Wd2 = 1, 1
        else:
            raise ValueError('Invalid weighting method')
        return np.diag(np.array([Wd1, Wd2]))
    
    def reg(v, a=0, o=2):
        return a*linalg.norm(v, o)
    
    cost = np.dot(weighting(), loss())
    return linalg.norm(cost,2) + reg(variables, a=lambda_reg)

def jacobian(variables, *args):
    return Jacobian(lambda x: objective(x, *args))(variables).ravel()

def hessian(variables, *args):
    return Hessian(lambda x: objective(x, *args))(variables)

bounds = [(0,1),(R_min,R_max)]
constr = (optimize.NonlinearConstraint(lambda x: x[0], lb=bounds[0][0], ub=bounds[0][1]),
          optimize.NonlinearConstraint(lambda x: x[1], lb=bounds[1][0], ub=bounds[1][1]))

xall, xhist  = {}, []
def callback(x, *args):
    xhist.append(x)

sol = []
for i, row in well1.iterrows():
    Rv_value       = row['Rv']
    Rh_value       = row['Rh']
    Csh_lin_value  = row['Csh_lin']
    GR_value       = row['GR']
    CALI_value     = row['CALI']
    Rvsh_value     = row['Rvsh_win'] #2.8133
    Rhsh_value     = row['Rhsh_win'] #0.7746

    x0 = (Csh_lin_value, np.mean([Rv_value, Rh_value]))
    c0 = (Rv_value, Rh_value, Rvsh_value, Rhsh_value, GR_value, CALI_value)
    res = optimize.minimize(objective,
                            x0          = x0,
                            args        = c0,
                            method      = 'L-BFGS-B',
                            #jac         = jacobian,
                            #hess        = hessian,
                            bounds      = bounds,
                            #constraints = constr,
                            tol         = 1e-6,
                            callback    = callback,
                            options     = {'maxiter':100},
                            )
    Csh_pred, Rss_pred = res.x
    sol.append({'Rv':Rv_value, 'Rh':Rh_value, 
                'Csh_pred':Csh_pred, 'Rss_pred':Rss_pred,
                'fun':res.fun, 'nfev':res.nfev, 'jac_norm':linalg.norm(res.jac),
                'Rv_sim':Csh_pred*Rvsh_value + (1-Csh_pred)*Rss_pred,
                'Rh_sim':1/(Csh_pred/Rhsh_value + (1-Csh_pred)/Rss_pred)})
    xall[i] = np.array([x0]+xhist)

In [ ]:
# collect and visualize results
well1_gb = pd.DataFrame(sol, index=well1.index)
well1_sol = well1.merge(well1_gb)
well1_sol.index = well1.index

error_metrics(well1_sol)
plot_crossplot(well1_sol)

In [ ]:
# visualize dataset
fig, axs = plt.subplots(1, 5, figsize=(10,10), sharey=True)
ax1a, ax2a, ax3a, ax4a, ax5a = axs

plot_curve(ax1a, well1_sol, 'GR', units='API', lb=0, ub=150, color='green')

ax2b, ax2c = ax2a.twiny(), ax2a.twiny()
plot_curve(ax2a, well1_sol, 'Csh_lin', units='v/v', lb=-0.05, ub=1.05, color='gray', ls='--')
plot_curve(ax2b, well1_sol, 'Csh_q', units='v/v', lb=-0.05, ub=1.05, color='gray', pad=8)
plot_curve(ax2c, well1_sol, 'Csh_pred', units='v/v', lb=-0.05, ub=1.05, color='r', pad=16)

ax3b, ax3c, ax3d = ax3a.twiny(), ax3a.twiny(), ax3a.twiny()
plot_curve(ax3a, well1_sol, 'AT10', units='ohm.m', lb=0.2, ub=100, color='b', semilog=True)
plot_curve(ax3b, well1_sol, 'AT30', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=8)
plot_curve(ax3c, well1_sol, 'AT60', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=16)
plot_curve(ax3d, well1_sol, 'AT90', units='ohm.m', lb=0.2, ub=100, color='r', semilog=True, pad=24)

ax4b = ax4a.twiny()
plot_curve(ax4a, well1_sol, 'Rss_q', units='ohm.m', lb=0.2, ub=100, color='gray', semilog=True)
plot_curve(ax4b, well1_sol, 'Rss_pred', units='ohm.m', lb=0.2, ub=100, color='r', semilog=True, pad=8)

ax5b, ax5c, ax5d = ax5a.twiny(), ax5a.twiny(), ax5a.twiny()
plot_curve(ax5a, well1_sol, 'Rv', units='ohm.m', lb=0.2, ub=100, color='darkred', semilog=True)
plot_curve(ax5b, well1_sol, 'Rh', units='ohm.m', lb=0.2, ub=100, color='darkblue', semilog=True, pad=8)
plot_curve(ax5c, well1_sol, 'Rv_sim', units='ohm.m', lb=0.2, ub=100, color='k', semilog=True, ls='--', pad=16)
plot_curve(ax5d, well1_sol, 'Rh_sim', units='ohm.m', lb=0.2, ub=100, color='k', semilog=True, ls='--', pad=24)

ax1a.invert_yaxis()
ax1a.set_ylabel('Depth [ft]', weight='bold')
plt.tight_layout()
plt.show()

***
## Field Dataset 2

In [ ]:
# load dataset
well2 = lasio.read('cases/well2.las').df()
well2_cols = ['HCAL','AT10','AT30','AT60','AT90','HCGR','PEFZ','TNPH','RHOZ','RV72_1DF','RH72_1DF']
well2 = well2[well2_cols]

In [ ]:
# rename columns to standard
well2_newcols = ['CALI','AT10','AT30','AT60','AT90','GR','PEF','NPHI','RHOZ','Rv','Rh']
well2.columns = well2_newcols

In [ ]:
# define baseline shale parameters
well2.loc[:,'Rvsh'] = 2.78
well2.loc[:,'Rhsh'] = 0.58

In [ ]:
# define starting and ending depth potints
zstart = int(np.argwhere(well2.index == 6292.75).squeeze())
zend   = int(np.argwhere(well2.index == 9078.25).squeeze())
well2  = well2.iloc[zstart:zend]
well2  = well2.dropna()


In [ ]:
# visualize dataset
fig, axs = plt.subplots(1, 5, figsize=(10,10), sharey=True)
ax1a, ax2a, ax3a, ax4a, ax5a = axs

ax1b = ax1a.twiny()
plot_curve(ax1a, well2, 'CALI', units='in', lb=0, ub=48, color='dodgerblue', alpha=0.33, fill=True)
plot_curve(ax1b, well2, 'GR', units='API', lb=0, ub=150, color='green', pad=8)

ax2b = ax2a.twiny()
plot_curve(ax2a, well2, 'NPHI', units='v/v', lb=0.6, ub=0.0, color='blue')
plot_curve(ax2b, well2, 'RHOZ', units='g/cm3', lb=1.65, ub=2.65, color='r', pad=8)

ax3b, ax3c, ax3d = ax3a.twiny(), ax3a.twiny(), ax3a.twiny()
plot_curve(ax3a, well2, 'AT10', units='ohm.m', lb=0.2, ub=100, color='b', semilog=True)
plot_curve(ax3b, well2, 'AT30', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=8)
plot_curve(ax3c, well2, 'AT60', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=16)
plot_curve(ax3d, well2, 'AT90', units='ohm.m', lb=0.2, ub=100, color='r', semilog=True, pad=24)

plot_curve(ax4a, well2, 'PEF', units='b/e', lb=0, ub=10, color='purple')

ax5b = ax5a.twiny()
plot_curve(ax5a, well2, 'Rv', units='ohm.m', lb=0.2, ub=100, color='darkred', semilog=True)
plot_curve(ax5b, well2, 'Rh', units='ohm.m', lb=0.2, ub=100, color='darkblue', semilog=True, pad=8)

ax1a.invert_yaxis()
ax1a.set_ylabel('Depth [ft]', weight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# linear (csh only) and quadratic inversion
well2['Csh_lin'] = (well2['GR'] - well2['GR'].min()) / (well2['GR'].max() - well2['GR'].min())
qinv = np.abs(quadratic_inversion(well2))
well2 = well2.join(qinv)

In [ ]:
# moving window shale properties
well2['Rvsh_win'] = np.nan_to_num(np.array(pd.Series(well2['Rv']).rolling(250).max().tolist()), nan=well2['Rv'].mean())
well2['Rhsh_win'] = np.nan_to_num(np.array(pd.Series(well2['Rh']).rolling(250).max().tolist()), nan=well2['Rh'].mean())
R_min = np.min([well2['Rh'].min(), well2['Rv'].min()])
R_max = np.max([well2['Rh'].max(), well2['Rv'].max()])

In [ ]:
# gradient-based inversion
grmax, calimax = well2['GR'].max(), well2['CALI'].max()
lambda_reg = 1e-5
possible_methods = ['L-BFGS-B', 'TNC', 'SLSQP', 'trust-constr']

In [ ]:
def objective(variables, *args):
    Csh, Rs = variables   
    Rv, Rh, Rvsh, Rhsh = args[0], args[1], args[2], args[3]
    grw, caliw = args[4]/grmax, args[5]/calimax
    
    def loss():
        eq1 = (Csh*Rvsh + (1-Csh)*Rs) - Rv
        eq2 = (Csh/Rhsh + (1-Csh)/Rs) - (1/Rh)
        return np.expand_dims(np.array([eq1, eq2]),-1)
    
    def weighting(method='gr'):
        if method=='gr':
            Wd1, Wd2 = 1/Rv/grw, 1*Rh/grw
        elif method=='cali':
            Wd1, Wd2 = 1/Rv/caliw, 1*Rh/caliw
        elif method=='data':
            Wd1, Wd2 = 1/Rv, 1*Rh
        elif method=='none':
            Wd1, Wd2 = 1, 1
        else:
            raise ValueError('Invalid weighting method')
        return np.diag(np.array([Wd1, Wd2]))
    
    def reg(v, a=0, o=2):
        return a*linalg.norm(v, o)
    
    cost = np.dot(weighting(), loss())
    return linalg.norm(cost,2) + reg(variables, a=lambda_reg)

def jacobian(variables, *args):
    return Jacobian(lambda x: objective(x, *args))(variables).ravel()

def hessian(variables, *args):
    return Hessian(lambda x: objective(x, *args))(variables)

bounds = [(0,1),(R_min,R_max)]
constr = (optimize.NonlinearConstraint(lambda x: x[0], lb=bounds[0][0], ub=bounds[0][1]),
          optimize.NonlinearConstraint(lambda x: x[1], lb=bounds[1][0], ub=bounds[1][1]))

xall, xhist  = {}, []
def callback(x, *args):
    xhist.append(x)

sol = []
for i, row in well2.iterrows():
    Rv_value       = row['Rv']
    Rh_value       = row['Rh']
    Csh_lin_value  = row['Csh_lin']
    GR_value       = row['GR']
    CALI_value     = row['CALI']
    Rvsh_value     = row['Rvsh_win'] #2.78
    Rhsh_value     = row['Rhsh_win'] #0.58
    
    x0 = (Csh_lin_value, np.mean([Rv_value, Rh_value]))
    c0 = (Rv_value, Rh_value, Rvsh_value, Rhsh_value, GR_value, CALI_value)
    res = optimize.minimize(objective,
                            x0          = x0,
                            args        = c0,
                            method      = 'L-BFGS-B',
                            #jac         = jacobian,
                            #hess        = hessian,
                            bounds      = bounds,
                            #constraints = constr,
                            tol         = 1e-6,
                            callback    = callback,
                            options     = {'maxiter':100},
                            )
    Csh_pred, Rss_pred = res.x
    sol.append({'Rv':Rv_value, 'Rh':Rh_value, 
                'Csh_pred':Csh_pred, 'Rss_pred':Rss_pred,
                'fun':res.fun, 'nfev':res.nfev, 'jac_norm':linalg.norm(res.jac),
                'Rv_sim':Csh_pred*Rvsh_value + (1-Csh_pred)*Rss_pred,
                'Rh_sim':1/(Csh_pred/Rhsh_value + (1-Csh_pred)/Rss_pred)})
    xall[i] = np.array([x0]+xhist)

In [ ]:
# collect and visualize results
well2_gb = pd.DataFrame(sol, index=well2.index)
well2_sol = pd.DataFrame(np.concatenate([well2_gb.values[:,2:], well2.values], axis=1), columns=well2_gb.columns[2:].tolist()+well2.columns.tolist())
well2_sol.index = well2.index

error_metrics(well2_sol)
plot_crossplot(well2_sol)

In [ ]:
# visualize dataset
fig, axs = plt.subplots(1, 5, figsize=(10,10), sharey=True)
ax1a, ax2a, ax3a, ax4a, ax5a = axs

plot_curve(ax1a, well2_sol, 'GR', units='API', lb=0, ub=150, color='green')

ax2b, ax2c = ax2a.twiny(), ax2a.twiny()
plot_curve(ax2a, well2_sol, 'Csh_lin', units='v/v', lb=-0.05, ub=1.05, color='gray', ls='--')
plot_curve(ax2b, well2_sol, 'Csh_q', units='v/v', lb=-0.05, ub=1.05, color='gray', pad=8)
plot_curve(ax2c, well2_sol, 'Csh_pred', units='v/v', lb=-0.05, ub=1.05, color='r', pad=16)

ax3b, ax3c, ax3d = ax3a.twiny(), ax3a.twiny(), ax3a.twiny()
plot_curve(ax3a, well2_sol, 'AT10', units='ohm.m', lb=0.2, ub=100, color='b', semilog=True)
plot_curve(ax3b, well2_sol, 'AT30', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=8)
plot_curve(ax3c, well2_sol, 'AT60', units='ohm.m', lb=0.2, ub=100, color='k', ls='--', semilog=True, pad=16)
plot_curve(ax3d, well2_sol, 'AT90', units='ohm.m', lb=0.2, ub=100, color='r', semilog=True, pad=24)

ax4b = ax4a.twiny()
plot_curve(ax4a, well2_sol, 'Rss_q', units='ohm.m', lb=0.2, ub=100, color='gray', semilog=True)
plot_curve(ax4b, well2_sol, 'Rss_pred', units='ohm.m', lb=0.2, ub=100, color='r', semilog=True, pad=8)

ax5b, ax5c, ax5d = ax5a.twiny(), ax5a.twiny(), ax5a.twiny()
plot_curve(ax5a, well2_sol, 'Rv', units='ohm.m', lb=0.2, ub=100, color='darkred', semilog=True)
plot_curve(ax5b, well2_sol, 'Rh', units='ohm.m', lb=0.2, ub=100, color='darkblue', semilog=True, pad=8)
plot_curve(ax5c, well2_sol, 'Rv_sim', units='ohm.m', lb=0.2, ub=100, color='k', semilog=True, ls='--', pad=16)
plot_curve(ax5d, well2_sol, 'Rh_sim', units='ohm.m', lb=0.2, ub=100, color='k', semilog=True, ls='--', pad=24)

ax1a.invert_yaxis()
ax1a.set_ylabel('Depth [ft]', weight='bold')
plt.tight_layout()
plt.show()

***
# END